# "MNIST Classification"
> "Classification of number images in MNIST Dataset"

- toc: false
- branch: master
- badges: true
- comments: true
- categories: [jupyter, pytorch, pytorch-lightning]
- hide: false
- search_exclude: true

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torch.utils.data import DataLoader
from torchvision import transforms, datasets
import pytorch_lightning as pl
from sklearn.model_selection import train_test_split
import numpy as np

In [2]:
class MNISTDm(pl.LightningDataModule):
    def __init__(self):
        super().__init__()
        self.train_ds = datasets.MNIST('', train=True, download=True, 
            transform=transforms.Compose([transforms.ToTensor()])
        )
        
        self.val_ds = datasets.MNIST('', train=False, download=True,
            transform=transforms.Compose([transforms.ToTensor()])
        )
        self.val_ds, self.test_ds = train_test_split(self.val_ds, 
                                                     test_size = 0.2)
        
        
    def train_dataloader(self):
        return DataLoader(self.train_ds, batch_size=128, shuffle=True)
        
    def val_dataloader(self):
        return DataLoader(self.val_ds, batch_size=128, shuffle=False)
    
    def test_dataloader(self):
        return DataLoader(self.test_ds, batch_size=128, shuffle=False)
    
dm = MNISTDm()
a = next(iter(dm.test_dataloader()))

In [3]:
class ConvNet(pl.LightningModule):
    def conv_layer(self, ni, no):
        return nn.Sequential(
            nn.Conv2d(ni, no, 3),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),
            nn.Dropout(0.2)
        )
    
    def __init__(self):
        super().__init__()
        self.backbone = nn.Sequential(
            self.conv_layer(1, 32),
            self.conv_layer(32, 64),
            self.conv_layer(64, 128),
            nn.AdaptiveAvgPool2d(1)
        )

        self.classifier = nn.Sequential(
            nn.Linear(128,64),
            nn.ReLU(inplace=True),
            nn.Linear(64, 10)
        )
        self.loss_fn = nn.CrossEntropyLoss()
            
    def forward(self, x):
        x = self.backbone(x)
        x = x.view(-1, 128)
        x = self.classifier(x)
        return x
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        matches = [torch.argmax(i) == j for i,j in zip(y_hat,y)]
        acc = matches.count(True)/len(matches)
        loss = self.loss_fn(y_hat, y)
        self.log('acc', acc, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        return {'loss':loss, 'acc':acc}
        
    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        matches = [torch.argmax(i) == j for i,j in zip(y_hat,y)]
        val_acc = matches.count(True)/len(matches)
        val_loss = self.loss_fn(y_hat, y)
        self.log('val acc', val_acc, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        return {'val loss':val_loss, 'val acc':val_acc}
        
    def configure_optimizers(self):
        return optim.Adam(self.parameters(), lr=1e-3)
    
    def predict(self, test):
        if len(test) == 1:
            test = test[None]
            pred = self(test)
            return (torch.argmax(pred)).item()
        else:
            pred = self(test)
            return (torch.argmax(pred, dim=1))
        
    def evaluate(self, testx, labels):
        preds = self.predict(testx)
        if isinstance(preds, int):
            return (preds==labels).float().item()
        else:
            matches = (preds==labels)
            acc = sum(matches)/len(matches)
            return acc.item()

In [4]:
if __name__ == '__main__':
    net = ConvNet()
    trainer = pl.Trainer(max_epochs=1, gpus=[0])
    trainer.fit(net, dm)

GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name       | Type             | Params
------------------------------------------------
0 | backbone   | Sequential       | 92.7 K
1 | classifier | Sequential       | 8.9 K 
2 | loss_fn    | CrossEntropyLoss | 0     
------------------------------------------------
101 K     Trainable params
0         Non-trainable params
101 K     Total params


In [7]:
testbatch = next(iter(dm.test_dataloader()))
batchx, batchy = testbatch
rnd = np.random.randint(0,len(batchx))
testx = testbatch[0][rnd]
testy = testbatch[1][rnd]
 
# pred = net.predict(testx) 
# acc = net.evaluate(testx, testy)

pred = net.predict(batchx)  #Predicting a batch
acc = net.evaluate(batchx, batchy)  #Evaluating a batch
acc

0.9609375